<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/75-soa/fer2013-fit_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.lib.io import file_io

#import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.keras.callbacks import ReduceLROnPlateau

#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten
#from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
#from keras.optimizers import SGD
#from keras.callbacks import ReduceLROnPlateau

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
def get_datagen(dataset):
    # Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches) indefinitely
    # rescale:          Rescaling factor (defaults to None). Multiply the data by the value provided (before applying any other transformation)
    # rotation_range:   Int. Degree range for random rotations
    # shear_range:      Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    # zoom_range:       Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range]
    # fill_mode :       Points outside the boundaries of the input are filled according to the given mode: {"constant", "nearest", "reflect" or "wrap"}
    # horizontal_flip:  Boolean. Randomly flip inputs horizontally
    datagen = ImageDataGenerator(
                        rescale=1./255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

    return datagen.flow_from_directory(
            dataset,
            target_size=(48, 48),
            color_mode='grayscale',
            shuffle = False,
            class_mode='categorical',
            batch_size=32)

In [0]:
#! unzip '/content/drive/My Drive/cs230 project/dataset/midha/test.zip'
#! unzip -q '/content/drive/My Drive/cs230 project/dataset/midha/train.zip'

In [4]:
%%bash
root='/content/train/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

0 angry: 4656
1 disgust: 699
2 fear: 4796
3 happy: 8028
4 sad: 5154
5 surprise: 3456
6 neutral: 5682


In [7]:
# FER2013 has 35,887 labelled images, which are divided into 3589 test and 28709 train images.
X_train_gen  = get_datagen('/content/train')
X_dev_gen    = get_datagen('/content/test')
X_webcam_gen    = get_datagen('/content/drive/My Drive/cs230 project/dataset/webcam')

# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
# x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
#               it should have value 3
# y:            Labels
# batch_size:   Int (default: 32)
#train_generator = train_datagen.flow(X_train, Y_train,  batch_size  = BS)

Found 32471 images belonging to 7 classes.
Found 7825 images belonging to 7 classes.


In [0]:
EPOCHS = 100
BS = 128
DROPOUT_RATE = 0.4
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

In [0]:
# Implement below paper CPCPCPFF depth 5, 2.4m params
# http://openaccess.thecvf.com/content_cvpr_2016_workshops/w28/papers/Kim_Fusing_Aligned_and_CVPR_2016_paper.pdf
# Reference: https://arxiv.org/pdf/1612.02903.pdf
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
#model.add(Dropout(DROPOUT_RATE))
model.add(Conv2D(32, (4, 4), activation='relu',padding='same',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
#model.add(Dropout(DROPOUT_RATE))
model.add(Conv2D(64, (5, 5), activation='relu',padding='same',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Flatten())
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(DROPOUT_RATE))
model.add(Dense(7, activation='softmax',name='fcsoftmax'))

#TODO: weight decay of 0.0001...initial learning rate is set to 0.01 and reduced by a factor of 2 at every 25 epoch
SGD(lr=SGD_LEARNING_RATE,momentum=0.9, decay=SGD_DECAY, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
#rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)
rlrop = ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)

In [0]:
history = model.fit_generator(
    generator = X_train_gen,
    validation_data = X_dev_gen, 
    steps_per_epoch=len(X_train_gen.filenames) // BS,
    validation_steps=len(X_dev_gen.filenames) // BS,
    shuffle=True,
    callbacks=[rlrop],
    epochs=EPOCHS,
    use_multiprocessing=True)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 253 steps, validate for 61 steps
Epoch 1/100
253/253 [==============================] - 7s 29ms/step - loss: 1.2074 - accuracy: 0.5338 - val_loss: 1.6712 - val_accuracy: 0.3391
Epoch 2/100
253/253 [==============================] - 7s 29ms/step - loss: 1.1049 - accuracy: 0.5807 - val_loss: 1.4715 - val_accuracy: 0.4401
Epoch 3/100
253/253 [==============================] - 7s 29ms/step - loss: 1.2163 - accuracy: 0.5262 - val_loss: 1.4977 - val_accuracy: 0.4365
Epoch 4/100
253/253 [==============================] - 7s 28ms/step - loss: 1.1752 - accuracy: 0.5493 - val_loss: 1.6429 - val_accuracy: 0.3356
Epoch 5/100
253/253 [==============================] - 7s 28ms/step - loss: 1.1271 - accuracy: 0.5615 - val_loss: 1.4636 - val_accuracy: 0.4585
Epoch 6/100
253/253 [==============================] - 7s 28ms/step - loss: 1.1600 - accuracy: 0.5456 - val_loss: 1.6220 - val_accuracy: 0.3668
Epoch 7/100
253/253 [======================

Process Keras_worker_ForkPoolWorker-75:
Process Keras_worker_ForkPoolWorker-101:
Process Keras_worker_ForkPoolWorker-99:
Process Keras_worker_ForkPoolWorker-58:
Process Keras_worker_ForkPoolWorker-97:
Process Keras_worker_ForkPoolWorker-70:
Process Keras_worker_ForkPoolWorker-93:
Process Keras_worker_ForkPoolWorker-90:
Process Keras_worker_ForkPoolWorker-69:
Process Keras_worker_ForkPoolWorker-87:
Process Keras_worker_ForkPoolWorker-64:
Process Keras_worker_ForkPoolWorker-73:
Process Keras_worker_ForkPoolWorker-57:
Process Keras_worker_ForkPoolWorker-80:
Process Keras_worker_ForkPoolWorker-79:
Process Keras_worker_ForkPoolWorker-56:
Process Keras_worker_ForkPoolWorker-86:
Process Keras_worker_ForkPoolWorker-82:
Process Keras_worker_ForkPoolWorker-91:
Process Keras_worker_ForkPoolWorker-72:
Process Keras_worker_ForkPoolWorker-59:
Process Keras_worker_ForkPoolWorker-54:
Process Keras_worker_ForkPoolWorker-53:
Process Keras_worker_ForkPoolWorker-52:
Process Keras_worker_ForkPoolWorker-63:

In [0]:
model.evaluate(X_dev_gen)

In [0]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [0]:
# Save model to disk
lr_str = '-SGD_LR_%.5f' % SGD_LEARNING_RATE
epoch_str = '-EPOCHS_' + str(EPOCHS)
bs_str = '-BS_' + str(BS)
dropout_str = '-DROPOUT_' + str(DROPOUT_RATE)
test_acc = 'test_acc_%.3f' % results_test[1]
model.save('/content/drive/My Drive/cs230 project/models/soa' + lr_str + epoch_str + bs_str + dropout_str + test_acc + '.h5')

In [0]:
from sklearn.metrics import accuracy_score
# configure image data augmentation
datagen = ImageDataGenerator(horizontal_flip=True)

# make a prediction using test-time augmentation
def tta_prediction(datagen, model, image, n_examples):
	# convert image into dataset
	samples = np.expand_dims(image, 0)
	# prepare iterator
	it = datagen.flow(samples, batch_size=n_examples)
	# make predictions for each augmented image
	yhats = model.predict_generator(it, steps=n_examples, verbose=0)
	# sum across predictions
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	return np.argmax(summed)
 
 # evaluate a model on a dataset using test-time augmentation
def tta_evaluate_model(model, testX, testY):
	# configure image data augmentation
	datagen = ImageDataGenerator(horizontal_flip=True)
	# define the number of augmented images to generate per test set image
	n_examples_per_image = 7
	yhats = list()
	for i in range(len(testX)):
		# make augmented prediction
		yhat = tta_prediction(datagen, model, testX[i], n_examples_per_image)
		# store for evaluation
		yhats.append(yhat)
	# calculate accuracy
	testY_labels = np.argmax(testY, axis=1)
	acc = accuracy_score(testY_labels, yhats)
	return acc

In [0]:
print('\n# Evaluate on test data')
TTA_results_test = tta_evaluate_model(model, X_test, Y_test)
print('test loss, test acc:', results_test)
print('TTA test acc:', TTA_results_test)